In [1]:
class Event:
    def __init__(self, info):
        self.info = info
        self.url = info[0]
        self.thumbnail = info[1]
        self.title = info[2]
        self.date = info[3]
        self.id = info[0].split('/')[-1]
    def __repr__(self):
        return str(self.info)
    def view(self):
        import requests
        response = requests.get(self.url).text
        head, tail = response.split('<div class="blind">')
        self.img = head.split('<div>')[-1].split('<img src="')[1].split('"')[0]
        self.content = (tail.split('</div>')[0].strip().replace('\r\n', '\n')
                        .replace('<h2>', '# ').replace('</h2>', '')
                        .replace('<h3>', '## ').replace('</h3>', '')
                        .replace('<h4>', '\n**').replace('</h4>', '**\n')
                        .replace('<p>', '> ').replace('</p>', '')
                        .replace('<ul>', '').replace('</ul>', '')
                        .replace('<ol>', '').replace('</ol>', '')
                        .replace('\t', '').replace('<li>', '').replace('</li>', '')
                        .replace('&amp;', '&').replace('&lt;', '<').replace('&gt;', '>')
                        .replace('.\n\n', '.').replace(' \n\n', '')
                        .replace('\n> \n', '\n\n')
                        )


In [2]:
class EventBundle:
    def __init__(self):
        print(' : ', 'Initialized!')
    def connection_check(self, host='http://8.8.8.8'):
        import requests
        try:
            requests.head(host, timeout=1)
        except requests.ConnectionError:
            return False
        return True

    def getOngoingEvent(self, page=1):
        import requests
        response = requests.get('https://maplestory.nexon.com/News/Event/Ongoing?page='+str(page)).text
        return self._pretreatEventHTML(response)

    def getClosedEvent(self, page=1):
        import requests
        response = requests.get('https://maplestory.nexon.com/News/Event/Closed?page='+str(page)).text
        return self._pretreatEventHTML(response)
    def _pretreatEventHTML(self, raw_html):
        raw_event_htmls = raw_html.split('<div class="event_board">')[1].split('</ul>')[0].split('<li>')[1:]
        raw_event_htmls = [i.strip() for i in raw_event_htmls]
        raw_event_htmls = [i for i in raw_event_htmls if 'zzzzzzzz.png' not in i]
        raw_event_htmls = [i.replace('\r\n', '') for i in raw_event_htmls]
        events = [['https://maplestory.nexon.com/'+i.split('a href="')[1].split('"')[0],
                i.split('<img src="')[1].split('"')[0],
                i.split('<dd class="data">')[1].split('p>')[1].split('">')[1].split('</a')[0],
                i.split('<dd class="date">')[1].split('p>')[1].split('</')[0],
        ] for i in raw_event_htmls]
        events = [Event(event) for event in events]
        return events



In [3]:
handler = EventBundle()
if handler.connection_check('https://maplestory.nexon.com'):
    latest_event = handler.getOngoingEvent(page=1)[0]
    latest_event.view()
    print('이벤트 제목 : ', latest_event.title)
    print('이벤트 기간 : ', latest_event.date)
    print('이벤트 썸네일 : ', latest_event.thumbnail)
    print('이벤트 보기 url : ', latest_event.url)
    print('이벤트 내용 사진 : ', latest_event.img)

    print('\n === 이벤트 내용 ===')
    print(latest_event.content)

Initialized!
이벤트 제목 :  썬데이 메이플
이벤트 기간 :  2023.07.23 ~ 2023.07.23
이벤트 썸네일 :  https://file.nexon.com/NxFile/download/FileDownloader.aspx?oidFile=5701596925418018980
이벤트 보기 url :  https://maplestory.nexon.com//News/Event/Ongoing/800?page=1
이벤트 내용 사진 :  https://lwi.nexon.com/maplestory/2023/0721_board/93AD7424854BB905.jpg

 === 이벤트 내용 ===
# 썬데이 메이플
## 이벤트 기간
> 2023년 7월 23일(일) 오전 7시 ~ 오후 11시 30분

* 주문의 흔적 강화 비용 50% 할인
* 소울 조각 사용 시 위대한 소울 획득 확률 5배!
위대한 소울 획득 확률 1% → 5%

